In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab, sparse_to_doc
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect
from util import get_path
import pandas as pd

In [2]:
# date0='0709_1157'
# date1='0709_1557'
# date2='0709_1934'
date0='0701_1957'
date1='0701_2357'
date2='0702_0357'
date3='0702_0757'
date4='0702_1157'
date5='0702_1557'
date6='0702_1957'
date7='0702_2357'
plate=13
dates=[date0,date1]


In [3]:
mat_skels=[read_mat(get_path(date,plate,True))['skel'] for date in dates]

In [4]:
dic_skels=[dic_to_sparse(mat_skel) for mat_skel in mat_skels]

In [5]:
skeleton_docs=[sparse_to_doc(sparse_skel) for sparse_skel in dic_skels]

In [ ]:
skeleton_docs_aligned = [realign(skeleton_doc,skeleton_docs[0],2) for skeleton_doc in skeleton_docs]

In [10]:
for i, date in enumerate([date0,date1,date2,date3,date4,date5]):
    sparse.save_npz(f'Data/skeleton_{date}_{plate}_full',sparse.csc_matrix(skeleton_docs_aligned[i]))

In [4]:
skeleton_docs_aligned = [sparse.dok_matrix(sparse.load_npz(f'Data/skeleton_{date}_{plate}_full.npz')) for date in dates]

In [ ]:
graph_tabs=[from_sparse_to_graph(realign(skeleton_doc,skeleton_docs[0],2)) for skeleton_doc in skeleton_docs]

In [9]:
nx_graph_poss=[generate_nx_graph(from_sparse_to_graph(realign(skeleton_doc,skeleton_docs[0],2))) for skeleton_doc in skeleton_docs]
nx_graphs_aligned=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[prune_graph(nx_graph) for nx_graph in nx_graphs_aligned]

number_common_nodes_found : 2143 [[ 1.00000000e+00 -1.38777878e-17]
 [ 1.38777878e-17  1.00000000e+00]] [0.0000000e+00 1.8189894e-12]
100000
200000
300000
400000
500000
600000
866490 867437
105736 740
379 109
86 70
54 39
30 0
0 0
number_common_nodes_found : 477 [[ 9.99999998e-01  5.78173765e-05]
 [-5.78173765e-05  9.99999998e-01]] [14.83091045 -2.20804855]
100000
200000
300000
400000
500000
600000
number_common_nodes_found : 562 [[ 9.99999999e-01 -4.53206814e-05]
 [ 4.53206814e-05  9.99999999e-01]] [ 1.22085997 -1.27077265]
100000
200000
300000
400000
500000
600000
899840 901142
110361 838
515 225
198 169
43 40
32 0
0 0


In [14]:
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[prune_graph(nx_graph) for nx_graph in nx_graphs_aligned]

In [10]:
xbegin=14000
xend = 18000
ybegin = 8000
yend=16000
sub_nx_graphs = [sub_graph(nx_graph,poss_aligned[i],xbegin,xend,ybegin,yend) for i, nx_graph in enumerate(nx_graph_pruned)]
# sub_skeleton= [generate_skeleton(nx_graph,(xend-xbegin,yend-ybegin),(xbegin,ybegin)) for nx_graph in sub_nx_graphs]

In [13]:
downstream_graphs = []
downstream_pos = []
begin=len(dates)-2
downstream_graphs=[sub_nx_graphs[begin]]
downstream_poss=[poss_aligned[begin]]
for i in range (begin-1,0,-1):
    print("i=",i)
    new_graphs,new_poss = second_identification(sub_nx_graphs[i],downstream_graphs[0],poss_aligned[i],downstream_poss[0],50,downstream_graphs[1:],downstream_poss[1:],tolerance=30)
    downstream_graphs=new_graphs
    downstream_poss=new_poss

i= 3
number removed first phase 19
number removed second phase 36
116 0 [16887  8355]
116 1 [16893  8366]
i= 2
number removed first phase 13
number removed second phase 27
121 0 [17317  9088]
121 1 [17312  9091]
121 2 [17311  9107]
i= 1
number removed first phase 12
number removed second phase 32
119 0 [17318  9090]
119 1 [17317  9088]
119 2 [17312  9091]
119 3 [17311  9107]


In [65]:
degree3_nodes = [[node for node in nx_graph.nodes if nx_graph.degree(node)>=3] for nx_graph in downstream_graphs]

In [69]:
t=0
tp1=1
plot_t_tp1(degree3_nodes[t],degree3_nodes[tp1],downstream_poss[t],downstream_poss[tp1],sub_skeleton[t+1].todense(),sub_skeleton[tp1+1].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
downstream_graphs = []
downstream_pos = []
begin=len(dates)-1
downstream_graphs=[nx_graph_pruned[begin]]
downstream_poss=[poss_aligned[begin]]
for i in range (begin-1,-1,-1):
    print("i=",i)
    new_graphs,new_poss = second_identification(nx_graph_pruned[i],downstream_graphs[0],poss_aligned[i],downstream_poss[0],50,downstream_graphs[1:],downstream_poss[1:],tolerance=30)
    downstream_graphs=new_graphs
    downstream_poss=new_poss

i= 0
number removed first phase 78
number removed second phase 352


C:\Users\coren\Documents\WUR\MsC Thesis\code\node_id.py:301: RuntimeWarning: invalid value encountered in arccos
  competitor = np.arccos(np.dot(candidate_vectors[0],-candidate_vectors[1]))


2 0 [9661 6729]
2 1 [9658 6714]


In [11]:
nx_graph_pruned=downstream_graphs
poss_aligned=downstream_poss

In [17]:
skeleton_docs_cleaned= [generate_skeleton(nx_graph,skeleton_docs_aligned[i].shape) for i,nx_graph in enumerate(nx_graph_pruned)]

In [18]:
for i, date in enumerate(dates):
    sparse.save_npz(f'Data/skeleton_{date}_{plate}_full_clean',sparse.csc_matrix(skeleton_docs_cleaned[i]))

In [12]:
tabs_labeled = [from_nx_to_tab(nx_graph,poss_aligned[i]) for i, nx_graph in enumerate(nx_graph_pruned)]

In [13]:
tabs = [from_sparse_to_graph(skeleton) for skeleton in skeleton_docs_cleaned]

NameError: name 'skeleton_docs_cleaned' is not defined

In [14]:
tabs_labeled_matlab = [from_nx_to_tab_matlab(nx_graph,poss_aligned[i]) for i, nx_graph in enumerate(nx_graph_pruned)]

In [ ]:
for i, date in enumerate(dates):
    tabs_labeled_matlab[i].to_csv(get_path(date,plate,True,extension='_full_labeled_matlab.csv'))
    tabs_labeled[i].to_csv(get_path(date,plate,True,extension='_full_labeled.csv'))
#     tabs[i].to_csv(get_path(date,plate,True,extension='_full.csv'))
    

In [23]:
for i, date in enumerate(dates):
    tabs_labeled[i].to_csv(f'Data/graph_{date}_{plate}_full_labeled.csv')
    tabs[i].to_csv(f'Data/graph_{date}_{plate}_full.csv')